Ejercicio:

- Crear un modelo de ML con los datos del boston housing por ejemplo.
- El modelo tiene al menos:
    - Quitar alguna columna
    - Label encoder de alguna columna
    - Escalado
- Crear una API con 2 endpoints:
    - GET /predict -> Le pasamos una fila de datos como argumento (parametros o body, como querais)
    - Cada vez que el modelo predice, almacena la hora, los datos de entrada y las predicciones en una BBDD Sqlite3


In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import pickle

In [2]:
# Cargamos los datos
df = pd.read_csv('boston.csv', index_col=0)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
X = df.drop('target', axis=1)
y = df['target']

In [4]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
# Dividimos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
print(f"Shape X_train: {X_train.shape}")
print(f"Shape X_test: {X_test.shape}")

Shape X_train: (379, 13)
Shape X_test: (127, 13)


In [11]:
# Primera Transformación de los datos: Eliminar algunas columnas 
class columnDropTransform():
    def __init__(self, columns):
        '''
        columns: lista de columnas a eliminar
        '''
        self.columns = columns
    
    def fit(self, X, y=None):             
        return self
    
    def transform(self, X, y=None):
        return X.drop(self.columns, axis=1) 

In [12]:
# Comprobamos que funciona correctamente la transformación de datos 
#columnas = ['ZN', 'CHAS', 'RAD', 'B']
colum = columnDropTransform(columns=['ZN', 'CHAS', 'RAD', 'B'])
colum.transform(X_train)

,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,LSTAT
182,0.09103,2.46,0.4880,7.155,92.2,2.7006,193.0,17.8,4.82
155,3.53501,19.58,0.8710,6.152,82.6,1.7455,403.0,14.7,15.02
280,0.03578,3.33,0.4429,7.820,64.5,4.6947,216.0,14.9,3.76
126,0.38735,25.65,0.5810,5.613,95.6,1.7572,188.0,19.1,27.26
329,0.06724,3.24,0.4600,6.333,17.2,5.2146,430.0,16.9,7.34
...,...,...,...,...,...,...,...,...,...
106,0.17120,8.56,0.5200,5.836,91.9,2.2110,384.0,20.9,18.66
270,0.29916,6.96,0.4640,5.856,42.1,4.4290,223.0,18.6,13.00
348,0.01501,2.01,0.4350,6.635,29.7,8.3440,280.0,17.0,5.99
435,11.16040,18.10,0.7400,6.629,94.6,2.1247,666.0,20.2,23.27


In [13]:
# Segunda Transformación de los datos: Binzarizar una columna (convertir variables numéricas en variables binarias (0 o 1) en función de un umbral o criterio específico)
class columBinarizeTransform():
    def __init__(self, column, threshold):  
        '''
        column: columna a binarizar
        threshold: umbral para binarizar la columna
        '''        
        self.column = column
        self.threshold = threshold
    
    def fit(self, X, y=None):             
        return self
    
    def transform(self, X, y=None):
        X[self.column] = X[self.column].apply(lambda x: 1 if x >= self.threshold else 0)
        return X

In [14]:
# Comprobamos que funciona correctamente la transformación de datos 
columna = 'AGE'
umbral = 80
colum = columBinarizeTransform(columna, umbral)
colum.transform(X_train)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
182,0.09103,0.0,2.46,0.0,0.4880,7.155,1,2.7006,3.0,193.0,17.8,394.12,4.82
155,3.53501,0.0,19.58,1.0,0.8710,6.152,1,1.7455,5.0,403.0,14.7,88.01,15.02
280,0.03578,20.0,3.33,0.0,0.4429,7.820,0,4.6947,5.0,216.0,14.9,387.31,3.76
126,0.38735,0.0,25.65,0.0,0.5810,5.613,1,1.7572,2.0,188.0,19.1,359.29,27.26
329,0.06724,0.0,3.24,0.0,0.4600,6.333,0,5.2146,4.0,430.0,16.9,375.21,7.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.17120,0.0,8.56,0.0,0.5200,5.836,1,2.2110,5.0,384.0,20.9,395.67,18.66
270,0.29916,20.0,6.96,0.0,0.4640,5.856,0,4.4290,3.0,223.0,18.6,388.65,13.00
348,0.01501,80.0,2.01,0.0,0.4350,6.635,0,8.3440,4.0,280.0,17.0,390.94,5.99
435,11.16040,0.0,18.10,0.0,0.7400,6.629,1,2.1247,24.0,666.0,20.2,109.85,23.27


In [15]:
# Tercera Transformación de los datos: Escalar los datos (convertir los datos a una escala común) 
# Verificar que funciona correctamente el escalado
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)  

array([[-0.40679535, -0.50281197, -1.24707626, ..., -0.20515671,
         0.40451209, -1.07775629],
       [ 0.00799171, -0.50281197,  1.21828293, ..., -1.57625236,
        -3.09803734,  0.35116456],
       [-0.41344957,  0.36438137, -1.12179223, ..., -1.48779457,
         0.32659121, -1.22625199],
       ...,
       [-0.41595107,  2.96596138, -1.31187834, ..., -0.55898784,
         0.36812613, -0.91385066],
       [ 0.92638065, -0.50281197,  1.00515608, ...,  0.8563367 ,
        -2.84814129,  1.50690937],
       [-0.39020739, -0.50281197, -0.36864804, ...,  1.16593894,
        -3.29495634, -0.26383177]])

In [16]:
# Creamos el pipeline con las transformaciones y el modelo de ML 
columnas = ['ZN', 'CHAS', 'RAD', 'B']
columna = 'AGE'
umbral = 80

# Modelo de ML: se llama pipeline
pipeline = Pipeline([
    ('columnDropTransform', columnDropTransform(['ZN', 'CHAS', 'RAD', 'B'])),
    ('columBinarizeTransform', columBinarizeTransform('AGE', 80)),
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(random_state=42))
])

In [17]:
# Entrenamos el modelo
pipeline.fit(X_train, y_train)

Pipeline(steps=[('columnDropTransform',
                 <__main__.columnDropTransform object at 0x13babed30>),
                ('columBinarizeTransform',
                 <__main__.columBinarizeTransform object at 0x13bc2a820>),
                ('scaler', StandardScaler()),
                ('model', RandomForestRegressor(random_state=42))])

In [18]:
X_test.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
173,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04
274,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4.0,254.0,17.6,396.90,3.53
491,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07
72,0.09164,0.0,10.81,0.0,0.413,6.065,7.8,5.2873,4.0,305.0,19.2,390.91,5.52
452,5.09017,0.0,18.10,0.0,0.713,6.297,91.8,2.3682,24.0,666.0,20.2,385.09,17.27


In [19]:
# Predecimos con el modelo entrenado
y_pred = pipeline.predict(X_test)
y_pred

array([23.597, 30.723, 16.767, 23.135, 16.157, 21.412, 19.24 , 15.7  ,
       21.882, 20.539, 20.909, 19.62 ,  8.615, 20.988, 19.235, 25.242,
       19.165,  8.036, 45.362, 14.312, 23.91 , 23.809, 14.535, 23.575,
       14.855, 14.214, 21.758, 14.986, 20.905, 20.834, 20.105, 23.344,
       31.784, 20.65 , 15.346, 16.046, 34.966, 19.181, 20.737, 23.71 ,
       18.297, 29.683, 45.041, 19.281, 22.212, 13.467, 15.471, 23.51 ,
       17.503, 27.961, 21.357, 34.527, 15.751, 26.081, 45.31 , 22.066,
       15.364, 32.022, 22.077, 19.452, 25.324, 33.669, 29.045, 18.834,
       26.607, 19.252, 13.573, 23.034, 28.203, 18.016, 20.234, 27.016,
        9.877, 21.468, 21.4  ,  7.354, 20.469, 45.689, 11.06 , 14.006,
       21.921, 11.545, 20.553,  8.827, 20.802, 27.34 , 16.35 , 23.316,
       24.199, 17.476, 21.991,  7.786, 18.496, 19.237, 23.336, 19.787,
       40.443, 11.448, 13.093, 11.995, 20.148, 23.455, 13.721, 19.838,
       21.249, 12.752, 19.047, 24.386, 20.224, 23.642,  8.817, 14.667,
      

In [20]:
# Evaluamos el modelo con el conjunto de test (score)
pipeline.score(X_test, y_test)

# Mostrar todas las predicciones
pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

,y_test,y_pred
173,23.6,23.597
274,32.4,30.723
491,13.6,16.767
72,22.8,23.135
452,16.1,16.157
...,...,...
418,8.8,8.541
117,19.2,20.458
42,25.3,23.899
322,20.4,21.635


In [21]:
# Guardamos el modelo
pickle.dump(pipeline, open('modelo_boston.pkl', 'wb'))

In [22]:
# Cargamos el modelo
modelo = pickle.load(open('modelo_boston.pkl', 'rb'))

In [23]:
X_test.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
173,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04
274,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4.0,254.0,17.6,396.90,3.53
491,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07
72,0.09164,0.0,10.81,0.0,0.413,6.065,7.8,5.2873,4.0,305.0,19.2,390.91,5.52
452,5.09017,0.0,18.10,0.0,0.713,6.297,91.8,2.3682,24.0,666.0,20.2,385.09,17.27


In [24]:
# Predecimos con el modelo cargado
y_pred = modelo.predict(X_test)
y_pred

# Evaluamos el modelo con el conjunto de test (score)
modelo.score(X_test, y_test)

0.8553694561066725

Conexión BD SQLite3:

In [25]:
# Importar la librería sqlite3
import sqlite3

In [42]:
# Conectar con la base de datos boston.db
conn = sqlite3.connect('boston.db')

# Crear el cursor
c = conn.cursor()


In [30]:
# Crear la tabla boston 
c.execute("""CREATE TABLE boston (
            CRIM FLOAT,
            INDUS FLOAT,
            NOX FLOAT,
            RM FLOAT,
            AGE FLOAT,
            DIS FLOAT,
            TAX FLOAT,
            PTRATIO FLOAT,
            LSTAT FLOAT,
            date TIMESTAMP,
            target FLOAT
            )""")
conn.commit()

In [31]:
# Verificar que la tabla se ha creado correctamente 
c.execute("""SELECT * FROM boston""")
c.fetchall()

[]

In [43]:
pd.read_sql_query("SELECT * FROM boston", conn) 

,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,LSTAT,date,target


In [34]:
y_pred

array([23.597, 30.723, 16.767, 23.135, 16.157, 21.412, 19.24 , 15.7  ,
       21.882, 20.539, 20.909, 19.62 ,  8.615, 20.988, 19.235, 25.242,
       19.165,  8.036, 45.362, 14.312, 23.91 , 23.809, 14.535, 23.575,
       14.855, 14.214, 21.758, 14.986, 20.905, 20.834, 20.105, 23.344,
       31.784, 20.65 , 15.346, 16.046, 34.966, 19.181, 20.737, 23.71 ,
       18.297, 29.683, 45.041, 19.281, 22.212, 13.467, 15.471, 23.51 ,
       17.503, 27.961, 21.357, 34.527, 15.751, 26.081, 45.31 , 22.066,
       15.364, 32.022, 22.077, 19.452, 25.324, 33.669, 29.045, 18.834,
       26.607, 19.252, 13.573, 23.034, 28.203, 18.016, 20.234, 27.016,
        9.877, 21.468, 21.4  ,  7.354, 20.469, 45.689, 11.06 , 14.006,
       21.921, 11.545, 20.553,  8.827, 20.802, 27.34 , 16.35 , 23.316,
       24.199, 17.476, 21.991,  7.786, 18.496, 19.237, 23.336, 19.787,
       40.443, 11.448, 13.093, 11.995, 20.148, 23.455, 13.721, 19.838,
       21.249, 12.752, 19.047, 24.386, 20.224, 23.642,  8.817, 14.667,
      

In [44]:
# Insertar en la tabla boston la predicción realizada con el modelo cargado 

# Consulta SQL para insertar los datos
sql = """INSERT INTO boston 
            (CRIM, INDUS, NOX, RM, AGE, DIS, TAX, PTRATIO, LSTAT, date, target) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """

# Obtener la fecha actual
from datetime import datetime
fecha_actual = datetime.now()

# Insertar los datos en la tabla boston 
for i in range(len(X_test)):
    c.execute(sql, (X_test.iloc[i]['CRIM'], X_test.iloc[i]['INDUS'], X_test.iloc[i]['NOX'], X_test.iloc[i]['RM'], X_test.iloc[i]['AGE'], X_test.iloc[i]['DIS'], X_test.iloc[i]['TAX'], X_test.iloc[i]['PTRATIO'], X_test.iloc[i]['LSTAT'], fecha_actual, y_pred[i]))


# Guardar los cambios
conn.commit()

In [45]:
# Verificar que se han insertado los datos correctamente
pd.read_sql_query("SELECT * FROM boston", conn)


,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,LSTAT,date,target
0,0.09178,4.05,0.510,6.416,84.1,2.6463,296.0,16.6,9.04,2023-05-24 21:12:16.610137,23.597
1,0.05644,6.41,0.447,6.758,32.9,4.0776,254.0,17.6,3.53,2023-05-24 21:12:16.610137,30.723
2,0.10574,27.74,0.609,5.983,98.8,1.8681,711.0,20.1,18.07,2023-05-24 21:12:16.610137,16.767
3,0.09164,10.81,0.413,6.065,7.8,5.2873,305.0,19.2,5.52,2023-05-24 21:12:16.610137,23.135
4,5.09017,18.10,0.713,6.297,91.8,2.3682,666.0,20.2,17.27,2023-05-24 21:12:16.610137,16.157
...,...,...,...,...,...,...,...,...,...,...,...
122,73.53410,18.10,0.679,5.957,100.0,1.8026,666.0,20.2,20.62,2023-05-24 21:12:16.610137,8.541
123,0.15098,10.01,0.547,6.021,82.6,2.7474,432.0,17.8,10.30,2023-05-24 21:12:16.610137,20.458
124,0.14150,6.91,0.448,6.169,6.6,5.7209,233.0,17.9,5.81,2023-05-24 21:12:16.610137,23.899
125,0.35114,7.38,0.493,6.041,49.9,4.7211,287.0,19.6,7.70,2023-05-24 21:12:16.610137,21.635


In [38]:
# Eliminar los datos de la tabla boston
c.execute("""DELETE FROM boston""")
conn.commit()

In [39]:
# Verificar que se han eliminado los datos correctamente
pd.read_sql_query("SELECT * FROM boston", conn)


,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,LSTAT,date,target


In [40]:
# Cerrar la conexión con la base de datos
conn.close()

In [46]:
pd.read_sql_query("SELECT * FROM boston", conn)

,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,LSTAT,date,target
0,0.09178,4.05,0.510,6.416,84.1,2.6463,296.0,16.6,9.04,2023-05-24 21:12:16.610137,23.597
1,0.05644,6.41,0.447,6.758,32.9,4.0776,254.0,17.6,3.53,2023-05-24 21:12:16.610137,30.723
2,0.10574,27.74,0.609,5.983,98.8,1.8681,711.0,20.1,18.07,2023-05-24 21:12:16.610137,16.767
3,0.09164,10.81,0.413,6.065,7.8,5.2873,305.0,19.2,5.52,2023-05-24 21:12:16.610137,23.135
4,5.09017,18.10,0.713,6.297,91.8,2.3682,666.0,20.2,17.27,2023-05-24 21:12:16.610137,16.157
...,...,...,...,...,...,...,...,...,...,...,...
128,0.09178,4.05,0.510,6.416,84.1,2.6463,296.0,16.6,9.04,2023-05-24 21:33:30.287226,23.597
129,0.09178,4.05,0.510,6.416,84.1,2.6463,296.0,16.6,9.04,2023-05-24 21:35:18.880413,23.597
130,0.09178,4.05,0.510,6.416,84.1,2.6463,296.0,16.6,9.04,2023-05-24 21:37:23.052935,23.597
131,0.09178,4.05,0.510,6.416,84.1,2.6463,296.0,16.6,9.04,2023-05-24 21:38:28.824150,23.597
